In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/My Drive/Grade 4-2/ECE59500 ML/Project

/content/drive/My Drive/Grade 4-2/ECE59500 ML/Project


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [ ]:
import csv
import pandas as pd

In [ ]:
import os

In [ ]:
import six

In [ ]:
from sklearn.model_selection import train_test_split

## Data Import

In [ ]:
%cd fairface/

/content/drive/My Drive/Grade 4-2/ECE59500 ML/Project/fairface


In [ ]:
import regex as re

In [ ]:
from PIL import Image

In [ ]:
%cd val

/content/drive/My Drive/Grade 4-2/ECE59500 ML/Project/fairface/val


In [ ]:
data = os.listdir()

In [ ]:
data.sort(key=lambda x: int(re.sub('\D', '', x)))

In [ ]:
data_img = []
for file in data:
  img_read = Image.open(file)
  img_resize = img_read.resize((28, 28))
  img_resize = np.array(img_resize)
  if img_resize.ndim == 2:
    data_img.append(img_resize)
  elif img_resize.ndim == 3:
    data_img.append(img_resize[:, :, 0])
data_img = np.array(data_img)
data_img = data_img.astype("float32") / 255

In [ ]:
import pickle

In [ ]:
with open('data_img.pkl', 'wb') as data_pickle:
  pickle.dump(data_img, data_pickle)

In [ ]:
with open('data_img.pkl', 'rb') as data_pickle:
  data_img = pickle.load(data_pickle)

In [ ]:
with open("fairface_label_val.csv", "r") as csv_file:
  reader = csv.reader(csv_file, delimiter=',')
  data_csv = pd.DataFrame(reader)

In [ ]:
data_csv = data_csv.rename(columns={0: 'file ', 1: 'age', 2:'gender', 3:'race', 4:'service_test'})

In [ ]:
data_csv.drop(index=data_csv.index[0], axis=0, inplace=True)

In [ ]:
data_csv.reset_index(drop=True, inplace=True)

In [ ]:
data_csv.groupby('race').size()

race
Black              1556
East Asian         1550
Indian             1516
Latino_Hispanic    1623
Middle Eastern     1209
Southeast Asian    1415
White              2085
dtype: int64

## Experiment Configuraitons

In [ ]:
from collections import namedtuple

In [ ]:
Config = namedtuple('Config', [
    'reweight', 'lr', 'num_steps', 'random', 'ratio_weighted', 'nval', 'hard_mining', 'bsize'
])

exp_repo = dict()


def RegisterExp(name):
    def _decorator(f):
        exp_repo[name] = f
        return f

    return _decorator


LR = 0.005
NUM_STEPS = 2880


@RegisterExp('baseline')
def baseline_config():
    return Config(
        reweight=False,
        num_steps=NUM_STEPS * 2,
        lr=LR,
        random=False,
        ratio_weighted=False,
        hard_mining=False,
        bsize=100,
        nval=0)


@RegisterExp('hardmining')
def hardmining_config():
    return Config(
        reweight=False,
        num_steps=NUM_STEPS * 2,
        lr=LR,
        random=False,
        ratio_weighted=False,
        hard_mining=True,
        bsize=500,
        nval=0)


@RegisterExp('ratio')
def ratio_config():
    return Config(
        reweight=False,
        num_steps=NUM_STEPS * 2,
        lr=LR,
        random=False,
        ratio_weighted=True,
        hard_mining=False,
        bsize=100,
        nval=0)


@RegisterExp('random')
def dpfish_config():
    return Config(
        reweight=True,
        num_steps=NUM_STEPS * 2,
        lr=LR,
        random=True,
        ratio_weighted=False,
        hard_mining=False,
        bsize=100,
        nval=0)


@RegisterExp('autodiff')
def autodiff_config():
    return Config(
        reweight=True,
        num_steps=NUM_STEPS,
        lr=LR,
        random=False,
        ratio_weighted=False,
        hard_mining=False,
        bsize=100,
        nval=10)

## Helper Functions

In [ ]:
def get_acc(logits, y):
    prediction = tf.cast(tf.sigmoid(logits) > 0.5, tf.float32)
    return tf.reduce_mean(tf.cast(tf.equal(prediction, y), tf.float32))

In [ ]:
class DataSet(object):
    def __init__(self,
                 images,
                 labels,
                 seed=None):
        #Construct a DataSet.
        self._num_examples = images.shape[0]
        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
        self._indices = np.arange(self._num_examples)

    @property
    def images(self):
        return self._images.reshape(-1, 784)

    @property
    def labels(self):
        return self._labels

    @property
    def indices(self):
        return self._indices

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size, fake_data=False, shuffle=True):
        """Return the next `batch_size` examples (x, y, ind) from this data set.
        `x` is image [B, 28, 28, 1]. `y` is label [B, 10], `ind` is indices [B].
        """
        start = self._index_in_epoch
        # Shuffle for the first epoch
        if self._epochs_completed == 0 and start == 0 and shuffle:
            perm0 = np.arange(self._num_examples)
            np.random.shuffle(perm0)
            self._images = self.images[perm0]
            self._labels = self.labels[perm0]
            self._indices = self.indices[perm0]
        # Go to the next epoch
        if start + batch_size > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Get the rest examples in this epoch
            rest_num_examples = self._num_examples - start
            images_rest_part = self._images[start:self._num_examples]
            labels_rest_part = self._labels[start:self._num_examples]
            indices_rest_part = self._indices[start:self._num_examples]
            # Shuffle the data
            if shuffle:
                perm = np.arange(self._num_examples)
                np.random.shuffle(perm)
                self._images = self.images[perm]
                self._labels = self.labels[perm]
                self._indices = self.indices[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            images_new_part = self._images[start:end]
            labels_new_part = self._labels[start:end]
            indices_new_part = self._indices[start:end]
            return np.concatenate(
                (images_rest_part, images_new_part), axis=0), np.concatenate(
                    (labels_rest_part, labels_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self._images[start:end], self._labels[start:end]

In [ ]:
def get_model(inputs,
              labels,
              is_training=True,
              dtype=tf.float32,
              w_dict=None,
              ex_wts=None,
              reuse=None):
    """Builds a simple LeNet.
    
    :param inputs:            [Tensor]    Inputs.
    :param labels:            [Tensor]    Labels.
    :param is_training:       [bool]      Whether in training mode, default True.
    :param dtype:             [dtype]     Data type, default tf.float32.
    :param w_dict:            [dict]      Dictionary of weights, default None.
    :param ex_wts:            [Tensor]    Example weights placeholder, default None.
    :param reuse:             [bool]      Whether to reuse variables, default None.
    """

    if w_dict is None:
        w_dict = {}

    def _get_var(name, shape, dtype, initializer):
        key = tf.compat.v1.get_variable_scope().name + '/' + name
        if key in w_dict:
            return w_dict[key]
        else:
            var = tf.compat.v1.get_variable(name, shape, dtype, initializer=initializer)
            w_dict[key] = var
            return var

    with tf.compat.v1.variable_scope('Model', reuse=reuse):
        inputs_ = tf.cast(tf.reshape(inputs, [-1, 28, 28, 1]), dtype)
        labels = tf.cast(labels, dtype)

        w_init = tf.compat.v1.truncated_normal_initializer(stddev=0.1)
        w1 = _get_var('w1', [5, 5, 1, 16], dtype, initializer=w_init)    # [14, 14, 16]
        w2 = _get_var('w2', [5, 5, 16, 32], dtype, initializer=w_init)    # [7, 7, 32]
        w3 = _get_var('w3', [5, 5, 32, 64], dtype, initializer=w_init)    # [4, 4, 64]
        w4 = _get_var('w4', [1024, 100], dtype, initializer=w_init)
        w5 = _get_var('w5', [100, 1], dtype, initializer=w_init)

        b_init = tf.constant_initializer(0.0)
        b1 = _get_var('b1', [16], dtype, initializer=b_init)
        b2 = _get_var('b2', [32], dtype, initializer=b_init)
        b3 = _get_var('b3', [64], dtype, initializer=b_init)
        b4 = _get_var('b4', [100], dtype, initializer=b_init)
        b5 = _get_var('b5', [1], dtype, initializer=b_init)

        act = tf.nn.relu

        # Conv-1
        l0 = tf.identity(inputs_, name='l0')
        z1 = tf.add(tf.nn.conv2d(inputs_, w1, [1, 1, 1, 1], 'SAME'), b1, name='z1')
        l1 = act(tf.nn.max_pool(z1, [1, 3, 3, 1], [1, 2, 2, 1], 'SAME'), name='l1')

        # Conv-2
        z2 = tf.add(tf.nn.conv2d(l1, w2, [1, 1, 1, 1], 'SAME'), b2, name='z2')
        l2 = act(tf.nn.max_pool(z2, [1, 3, 3, 1], [1, 2, 2, 1], 'SAME'), name='l2')

        # Conv-3
        z3 = tf.add(tf.nn.conv2d(l2, w3, [1, 1, 1, 1], 'SAME'), b3, name='z3')
        l3 = act(tf.nn.max_pool(z3, [1, 3, 3, 1], [1, 2, 2, 1], 'SAME'), name='l3')

        # FC-4
        z4 = tf.add(tf.matmul(tf.reshape(l3, [-1, 1024]), w4), b4, name='z4')
        l4 = act(z4, name='l4')

        # FC-5
        z5 = tf.add(tf.matmul(l4, w5), b5, name='z5')

        logits = tf.squeeze(z5)
        out = tf.sigmoid(logits)
        if ex_wts is None:
            # Average loss.
            loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
        else:
            # Weighted loss.
            loss = tf.reduce_sum(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels) * ex_wts)
    return w_dict, loss, logits

In [ ]:
def evaluate(sess, x_, y_, acc_, train_set, test_set):
    # Calculate final results.
    acc_sum = 0.0
    acc_test_sum = 0.0
    train_bsize = 100
    for step in six.moves.xrange(3600 // train_bsize):
        x, y = train_set.next_batch(train_bsize)
        acc = sess.run(acc_, feed_dict={x_: x, y_: y})
        acc_sum += acc

    test_bsize = 100
    for step in six.moves.xrange(360 // test_bsize):
        x_test, y_test = test_set.next_batch(test_bsize)
        acc = sess.run(acc_, feed_dict={x_: x_test, y_: y_test})
        acc_test_sum += acc

    train_acc = acc_sum / float(3600 // train_bsize)
    test_acc = acc_test_sum / float(360 // test_bsize)
    return train_acc, test_acc

# Binary Classification

# Baseline

## Baseline Results - Balanced Data

In [ ]:
def get_balance_dataset(data_img, seed=0):
    rnd = np.random.RandomState(seed)

    pos_ratio=0.5
    ntrain=3600
    nval=exp_repo['baseline']().nval
    ntest=360
    seed=0

    ratio = 1 - pos_ratio
    ratio_test = 0.5

    class_poc = data_img[data_csv.index[data_csv['race'] != "White"], :, :]
    class_caucasian = data_img[data_csv.index[data_csv['race'] == "White"], :, :]

    class_poc_train = class_poc[0:1800, :, :]
    class_poc_test = class_poc[1800:1800+180, :, :]
    class_caucasian_train = class_caucasian[0:1800, :, :]
    class_caucasian_test = class_caucasian[1800:1800+180, :, :]

    x_train = np.concatenate((class_poc_train, class_caucasian_train), axis=0)
    y_train = np.concatenate((np.zeros(1800), np.ones(1800)), axis=0)
    x_test = np.concatenate((class_poc_test, class_caucasian_test), axis=0)
    y_test = np.concatenate((np.zeros(180), np.ones(180)), axis=0)

    train_set = DataSet(x_train * 255.0, y_train)
    train_poc_set = DataSet(class_poc_train* 255.0, np.zeros(1800))
    train_caucasian_set = DataSet(class_caucasian_train * 255.0, np.ones(1800))
    test_set = DataSet(x_test * 255.0, y_test)

    return train_set, test_set, train_caucasian_set, train_poc_set

In [ ]:
def run(dataset, exp_name, seed):
    folder = os.path.join('Binary classification: Balanced Data')
    if not os.path.exists(folder):
        os.makedirs(folder)

    with tf.Graph().as_default(), tf.compat.v1.Session() as sess:
        config = exp_repo[exp_name]()
        bsize = config.bsize
        train_set, test_set, train_pos_set, train_neg_set = get_balance_dataset(
            dataset, seed=seed)
        # if config.nval == 0:
        #     val_set = BalancedDataSet(train_pos_set, train_neg_set)
        x_ = tf.compat.v1.placeholder(tf.float32, [None, 784], name='x')
        y_ = tf.compat.v1.placeholder(tf.float32, [None], name='y')
        ex_wts_ = tf.compat.v1.placeholder(tf.float32, [None], name='ex_wts')
        lr_ = tf.compat.v1.placeholder(tf.float32, [], name='lr')

        # Build training model.
        with tf.name_scope('Train'):
            _, loss_c, logits_c = get_model(
                x_, y_, is_training=True, dtype=tf.float32, w_dict=None, ex_wts=ex_wts_, reuse=None)
            train_op = tf.compat.v1.train.MomentumOptimizer(config.lr, 0.9).minimize(loss_c)

        print(loss_c)

        # Build evaluation model.
        with tf.name_scope('Val'):
            _, loss_eval, logits_eval = get_model(
                x_,
                y_,
                is_training=False,
                dtype=tf.float32,
                w_dict=None,
                ex_wts=ex_wts_,
                reuse=True)
            acc_ = get_acc(logits_eval, y_)

        print(loss_eval)

        ex_weights_ = tf.ones([bsize], dtype=tf.float32) / float(bsize)
        tf.summary.create_file_writer('log_dir')


        lr = config.lr
        num_steps = config.num_steps

        acc_sum = 0.0
        acc_test_sum = 0.0
        loss_sum = 0.0
        count = 0
        sess.run(tf.compat.v1.global_variables_initializer())
        for step in six.moves.xrange(num_steps):
            x, y = train_set.next_batch(bsize)

            # Use 50% learning rate for the second half of training.
            if step > num_steps // 2:
                lr = config.lr / 2.0
            else:
                lr = config.lr

            ex_weights = sess.run(
                ex_weights_, feed_dict={x_: x,
                                        y_: y})
            loss, acc, _ = sess.run(
                [loss_c, acc_, train_op],
                feed_dict={
                    x_: x,
                    y_: y,
                    ex_wts_: ex_weights,
                    lr_: lr
                })
            if (step + 1) % 72 == 0:
                train_acc, test_acc = evaluate(sess, x_, y_, acc_, train_set, test_set)
                print(f'Step {step+1} Loss {loss} Train acc {train_acc} Test acc {test_acc}')
                tf.summary.create_file_writer('log_dir')
                acc_sum = 0.0
                loss_sum = 0.0
                acc_test_sum = 0.0
                count = 0

        # Final evaluation.
        train_acc, test_acc = evaluate(sess, x_, y_, acc_, train_set, test_set)
        print('Final', 'Train acc', train_acc, 'Test acc', test_acc)
    return train_acc, test_acc

In [ ]:
def run_many(dataset, nrun):
    train_acc_list = []
    test_acc_list = []
    for trial in range(nrun):
        train_acc, test_acc = run(
            dataset, 'baseline', (trial * 123456789) % 100000)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

    train_acc_list = np.array(train_acc_list)
    test_acc_list = np.array(test_acc_list)
    print('baseline', 'Train acc {:.3f}% ({:.3f}%)'.format(train_acc_list.mean() * 100.0,
                                                         train_acc_list.std() * 100.0))
    print('baseline', 'Test acc {:.3f}% ({:.3f}%)'.format(test_acc_list.mean() * 100.0,
                                                        test_acc_list.std() * 100.0))

In [ ]:
run_many(data_img, nrun=10)

Tensor("Train/Model/Sum:0", shape=(), dtype=float32)
Tensor("Val/Model/Sum:0", shape=(), dtype=float32)
Step 72 Loss 0.6937279105186462 Train acc 0.4999999933772617 Test acc 0.5
Step 144 Loss 0.6931649446487427 Train acc 0.4999999917215771 Test acc 0.4933333396911621
Step 216 Loss 0.6936889886856079 Train acc 0.4999999991721577 Test acc 0.48999998966852826
Step 288 Loss 0.6933047771453857 Train acc 0.4999999933772617 Test acc 0.4766666690508525
Step 360 Loss 0.6928186416625977 Train acc 0.499999994205104 Test acc 0.49333332975705463
Step 432 Loss 0.6931508779525757 Train acc 0.4999999966886308 Test acc 0.5000000099341074
Step 504 Loss 0.6933048963546753 Train acc 0.4999999966886308 Test acc 0.5
Step 576 Loss 0.693379819393158 Train acc 0.4999999966886308 Test acc 0.5300000011920929
Step 648 Loss 0.6942481994628906 Train acc 0.4999999983443154 Test acc 0.47999998927116394
Step 720 Loss 0.6931688189506531 Train acc 0.4999999991721577 Test acc 0.5166666706403097
Step 792 Loss 0.6924071311

KeyboardInterrupt: ignored

# Reweighted Model

In [ ]:
def get_imbalance_dataset(data_img, exp_name, seed, pos_ratio):
    ntrain=3600
    nval=exp_repo[exp_name]().nval
    ntest=360
    seed=0

    ratio = 1 - pos_ratio
    ratio_test = 0.5
    rnd = np.random.RandomState(seed)

    class_poc = data_img[data_csv.index[data_csv['race'] != "White"], :, :]
    class_caucasian = data_img[data_csv.index[data_csv['race'] == "White"], :, :]

    x_train_poc, x_test_poc = train_test_split(class_poc, test_size=0.1, random_state=25)
    x_train_caucasian, x_test_caucasian = train_test_split(class_caucasian, test_size=0.1, random_state=25)

    # First shuffle, negative.
    idx = np.arange(x_train_poc.shape[0])
    rnd.shuffle(idx)
    x_train_poc = x_train_poc[idx]

    nval_small_neg = int(np.floor(nval * ratio_test))
    ntrain_small_neg = int(np.floor(ntrain * ratio)) - nval_small_neg

    x_val_poc = x_train_poc[:nval_small_neg]    # 450 4 in validation.
    x_train_poc = x_train_poc[nval_small_neg:nval_small_neg + ntrain_small_neg]    # 500 4 in training.

    idx = np.arange(x_test_poc.shape[0])
    rnd.shuffle(idx)
    x_test_poc = x_test_poc[:int(np.floor(ntest * ratio_test))]    # 450 4 in testing.

    # First shuffle, positive.
    idx = np.arange(x_train_caucasian.shape[0])
    rnd.shuffle(idx)
    x_train_caucasian = x_train_caucasian[idx]

    nvalsmall_pos = int(np.floor(nval * (1 - ratio_test)))
    ntrainsmall_pos = int(np.floor(ntrain * (1 - ratio))) - nvalsmall_pos

    x_val_caucasian = x_train_caucasian[:nvalsmall_pos]    # 50 9 in validation.
    x_train_caucasian = x_train_caucasian[nvalsmall_pos:nvalsmall_pos + ntrainsmall_pos]    # 4500 9 in training.

    idx = np.arange(x_test_caucasian.shape[0])
    rnd.shuffle(idx)
    x_test_caucasian = x_test_caucasian[idx]
    x_test_caucasian = x_test_caucasian[:int(np.floor(ntest * (1 - ratio_test)))]    # 500 9 in testing.
    
    y_train_subset = np.concatenate([np.zeros([x_train_poc.shape[0]]), np.ones([x_train_caucasian.shape[0]])])
    y_val_subset = np.concatenate([np.zeros([x_val_poc.shape[0]]), np.ones([x_val_caucasian.shape[0]])])
    y_test_subset = np.concatenate([np.zeros([x_test_poc.shape[0]]), np.ones([x_test_caucasian.shape[0]])])

    y_train_pos_subset = np.ones([x_train_caucasian.shape[0]])
    y_train_neg_subset = np.zeros([x_train_poc.shape[0]])

    x_train_subset = np.concatenate([x_train_poc, x_train_caucasian], axis=0).reshape([-1, 28, 28, 1])
    x_val_subset = np.concatenate([x_val_poc, x_val_caucasian], axis=0).reshape([-1, 28, 28, 1])
    x_test_subset = np.concatenate([x_test_poc, x_test_caucasian], axis=0).reshape([-1, 28, 28, 1])

    x_train_pos_subset = x_train_caucasian.reshape([-1, 28, 28, 1])
    x_train_neg_subset = x_train_poc.reshape([-1, 28, 28, 1])

    # Final shuffle.
    idx = np.arange(x_train_subset.shape[0])
    rnd.shuffle(idx)
    x_train_subset = x_train_subset[idx]
    y_train_subset = y_train_subset[idx]

    idx = np.arange(x_val_subset.shape[0])
    rnd.shuffle(idx)
    x_val_subset = x_val_subset[idx]
    y_val_subset = y_val_subset[idx]

    idx = np.arange(x_test_subset.shape[0])
    rnd.shuffle(idx)
    x_test_subset = x_test_subset[idx]
    y_test_subset = y_test_subset[idx]

    train_set = DataSet(x_train_subset * 255.0, y_train_subset)
    train_pos_set = DataSet(x_train_pos_subset * 255.0, y_train_pos_subset)
    train_neg_set = DataSet(x_train_neg_subset * 255.0, y_train_neg_subset)
    val_set = DataSet(x_val_subset * 255.0, y_val_subset)
    test_set = DataSet(x_test_subset * 255.0, y_test_subset)

    return train_set, val_set, test_set, train_pos_set, train_neg_set

In [ ]:
def run_imbalance(dataset, exp_name, seed, pos_ratio):
    folder = os.path.join('Binary classification: Imbalanced Data')
    if not os.path.exists(folder):
        os.makedirs(folder)

    with tf.Graph().as_default(), tf.compat.v1.Session() as sess:
        config = exp_repo[exp_name]()
        bsize = config.bsize
        nval = config.nval
        train_set, val_set, test_set, train_pos_set, train_neg_set = get_imbalance_dataset(
            dataset, exp_name, seed, pos_ratio)
        x_ = tf.compat.v1.placeholder(tf.float32, [None, 784], name='x')
        y_ = tf.compat.v1.placeholder(tf.float32, [None], name='y')
        x_val_ = tf.compat.v1.placeholder(tf.float32, [None, 784], name='x_val')
        y_val_ = tf.compat.v1.placeholder(tf.float32, [None], name='y_val')
        ex_wts_ = tf.compat.v1.placeholder(tf.float32, [None], name='ex_wts')
        lr_ = tf.compat.v1.placeholder(tf.float32, [], name='lr')

        # Build training model.
        with tf.name_scope('Train'):
            _, loss_c, logits_c = get_model(
                x_, y_, is_training=True, dtype=tf.float32, w_dict=None, ex_wts=ex_wts_, reuse=None)
            train_op = tf.compat.v1.train.MomentumOptimizer(config.lr, 0.9).minimize(loss_c)

        # Build evaluation model.
        with tf.name_scope('Val'):
            _, loss_eval, logits_eval = get_model(
                x_,
                y_,
                is_training=False,
                dtype=tf.float32,
                w_dict=None,
                ex_wts=ex_wts_,
                reuse=True)
            acc_ = get_acc(logits_eval, y_)

        ex_weights_ = tf.ones([bsize], dtype=tf.float32) / float(bsize)
        tf.summary.create_file_writer('log_dir')

        if config.reweight:
            if config.random:
                ex_weights_ = reweight_random(bsize)
            else:
                ex_weights_ = reweight_autodiff(
                    x_,
                    y_,
                    x_val_,
                    y_val_,
                    bsize,
                    min(bsize, nval),
                    eps=0.0,
                    gate_gradients=1)
        else:
            if config.hard_mining:
                ex_weights_ = reweight_hard_mining(x_, y_, positive=True)
            else:
                if config.ratio_weighted:
                    # Weighted by the ratio of each class.
                    ex_weights_ = pos_ratio * (1 - y_) + (1 - pos_ratio) * (y_)
                else:
                    # Weighted by uniform.
                    ex_weights_ = tf.ones([bsize], dtype=tf.float32) / float(bsize)

        lr = config.lr
        num_steps = config.num_steps

        acc_sum = 0.0
        acc_test_sum = 0.0
        loss_sum = 0.0
        count = 0
        sess.run(tf.compat.v1.global_variables_initializer())
        for step in six.moves.xrange(num_steps):
            x, y = train_set.next_batch(bsize)
            x_val, y_val = val_set.next_batch(min(bsize, nval))

            # Use 50% learning rate for the second half of training.
            if step > num_steps // 2:
                lr = config.lr / 2.0
            else:
                lr = config.lr

            ex_weights = sess.run(
                ex_weights_, feed_dict={x_: x,
                                        y_: y,
                                        x_val_: x_val,
                                        y_val_: y_val})
            loss, acc, _ = sess.run(
                [loss_c, acc_, train_op],
                feed_dict={
                    x_: x,
                    y_: y,
                    x_val_: x_val,
                    y_val_: y_val,
                    ex_wts_: ex_weights,
                    lr_: lr
                })
            if (step + 1) % 72 == 0:
                train_acc, test_acc = evaluate(sess, x_, y_, acc_, train_set, test_set)
                print('Step', step + 1, 'Loss', loss, 'Train acc', train_acc, 'Test acc',
                          test_acc)
                acc_sum = 0.0
                loss_sum = 0.0
                acc_test_sum = 0.0
                count = 0

        # Final evaluation.
        train_acc, test_acc = evaluate(sess, x_, y_, acc_, train_set, test_set)
        print('Final', 'Train acc', train_acc, 'Test acc', test_acc)
    return train_acc, test_acc

In [ ]:
def run_many_imbalance(dataset, exp_name, nrun):
    results = []
    for pos_ratio in np.arange(0.6, 1, 0.1):
      train_acc_list = []
      test_acc_list = []
      for trial in range(nrun):
          train_acc, test_acc = run_imbalance(
              dataset, exp_name, (trial * 123456789) % 100000, pos_ratio)
          train_acc_list.append(train_acc)
          test_acc_list.append(test_acc)

      train_acc_list = np.array(train_acc_list)
      test_acc_list = np.array(test_acc_list)
      print(exp_name, pos_ratio, 'Train acc {:.3f}% ({:.3f}%)'.format(train_acc_list.mean() * 100.0,
                                                          train_acc_list.std() * 100.0))
      print(exp_name, pos_ratio, 'Test acc {:.3f}% ({:.3f}%)'.format(test_acc_list.mean() * 100.0,
                                                          test_acc_list.std() * 100.0))
      results.append([train_acc_list.mean() * 100.0, test_acc_list.mean() * 100.0])
    return results

In [ ]:
experiments = {'autodiff':[], 'hardmining':[], 'ratio':[], 'random':[], 'baseline':[]}
for exp_name in ['autodiff', 'hardmining', 'ratio', 'random', 'baseline']:
  experiments[exp_name] = run_many_imbalance(data_img, exp_name, nrun=1)

Step 72 Loss nan Train acc 0.4283333321412404 Test acc 0.476666659116745
Step 144 Loss nan Train acc 0.4324999981456333 Test acc 0.5233333309491476
Step 216 Loss nan Train acc 0.4366666649778684 Test acc 0.5299999912579855
Step 288 Loss nan Train acc 0.43611110912428963 Test acc 0.46666666865348816
Step 360 Loss nan Train acc 0.430277777214845 Test acc 0.4999999900658925
Step 432 Loss nan Train acc 0.4275000012583203 Test acc 0.5033333202203115
Step 504 Loss nan Train acc 0.43416666405068505 Test acc 0.503333330154419
Step 576 Loss nan Train acc 0.42944444302055573 Test acc 0.5033333202203115
Step 648 Loss nan Train acc 0.42916666385200286 Test acc 0.48333332935969037
Step 720 Loss nan Train acc 0.43888888508081436 Test acc 0.49666666984558105
Step 792 Loss nan Train acc 0.43583333326710594 Test acc 0.503333330154419
Step 864 Loss nan Train acc 0.4324999964899487 Test acc 0.5100000003973643
Step 936 Loss nan Train acc 0.42638888706763584 Test acc 0.5133333404858907
Step 1008 Loss nan T

In [ ]:
for exp_name in ['random', 'baseline']:
  run_many_imbalance(data_img, exp_name, nrun=1)

random
Step 72 Loss 0.29923537 Train acc 0.9213888926638497 Test acc 0.0
Step 144 Loss 0.20141357 Train acc 0.9211111184623506 Test acc 0.0
Step 216 Loss 0.3088569 Train acc 0.9202777793010076 Test acc 0.0
Step 288 Loss 0.26012236 Train acc 0.9202777793010076 Test acc 0.0
Step 360 Loss 0.47783712 Train acc 0.9252777778440051 Test acc 0.0
Step 432 Loss 0.18504608 Train acc 0.9200000017881393 Test acc 0.0
Step 504 Loss 0.3292215 Train acc 0.9249999953640832 Test acc 0.0
Step 576 Loss 0.15184894 Train acc 0.9188888900809817 Test acc 0.0
Step 648 Loss 0.15826714 Train acc 0.9194444467624029 Test acc 0.0
Step 720 Loss 0.20386183 Train acc 0.9213888893524805 Test acc 0.0
Step 792 Loss 0.20112236 Train acc 0.9188888917366663 Test acc 0.0
Step 864 Loss 0.28428543 Train acc 0.9233333385652966 Test acc 0.0
Step 936 Loss 0.29409814 Train acc 0.91916666759385 Test acc 0.0
Step 1008 Loss 0.2119308 Train acc 0.9202777809566922 Test acc 0.0
Step 1080 Loss 0.3704489 Train acc 0.9213888893524805 Test a

In [ ]:

def reweight_random(bsize, eps=0.0):
    """Reweight examples using random numbers.
    
    :param bsize:             [int]       Batch size.
    :param eps:               [float]     Minimum example weights, default 0.0.
    """
    ex_weight = tf.compat.v1.random_normal([bsize], mean=0.0, stddev=1.0)
    ex_weight_plus = tf.maximum(ex_weight, eps)
    ex_weight_sum = tf.reduce_sum(ex_weight_plus)
    ex_weight_sum += tf.cast(tf.equal(ex_weight_sum, 0.0), tf.float32)
    ex_weight_norm = ex_weight_plus / ex_weight_sum
    return ex_weight_norm

In [ ]:
def reweight_hard_mining(inp, label, positive=False):
    """Reweight examples using hard mining.
    
    :param inp:          [Tensor]   [N, ...] Inputs.
    :param label:        [Tensor]   [N] Labels
    :param positive:     [bool]     Whether perform hard positive mining or hard negative mining.
    :return              [Tensor]   Examples weights of the same shape as the first dim of inp.
    """
    _, loss, logits = get_model(inp, label, ex_wts=None, is_training=True, reuse=True)

    # Mine for positive
    if positive:
        loss_mask = loss * label
    else:
        loss_mask = loss * (1 - label)

    if positive:
        k = tf.cast(tf.reduce_sum(1 - label), tf.int32)
    else:
        k = tf.cast(tf.reduce_sum(label), tf.int32)
    k = tf.maximum(k, 1)
    loss_sorted, loss_sort_idx = tf.nn.top_k(loss_mask, k)

    if positive:
        mask = 1 - label
    else:
        mask = label

    updates = tf.ones([tf.shape(loss_sort_idx)[0]], dtype=label.dtype)
    mask_add = tf.scatter_nd(tf.expand_dims(loss_sort_idx, axis=1), updates, [tf.shape(inp)[0]])
    mask = tf.maximum(mask, mask_add)
    mask_sum = tf.reduce_sum(mask)
    mask_sum += tf.cast(tf.equal(mask_sum, 0.0), tf.float32)
    mask = mask / mask_sum
    return mask

In [ ]:
def reweight_autodiff(inp_a,
                      label_a,
                      inp_b,
                      label_b,
                      bsize_a,
                      bsize_b,
                      eps=0.0,
                      gate_gradients=1):
    """Reweight examples using automatic differentiation.
    :param inp_a:             [Tensor]    Inputs for the noisy pass.
    :param label_a:           [Tensor]    Labels for the noisy pass.
    :param inp_b:             [Tensor]    Inputs for the clean pass.
    :param label_b:           [Tensor]    Labels for the clean pass.
    :param bsize_a:           [int]       Batch size for the noisy pass.
    :param bsize_b:           [int]       Batch size for the clean pass.
    :param eps:               [float]     Minimum example weights, default 0.0.
    :param gate_gradients:    [int]       Tensorflow gate gradients, reduce concurrency.
    """
    ex_wts_a = tf.zeros([bsize_a], dtype=tf.float32)
    ex_wts_b = tf.ones([bsize_b], dtype=tf.float32) / float(bsize_b)
    w_dict, loss_a, logits_a = get_model(
        inp_a, label_a, ex_wts=ex_wts_a, is_training=True, reuse=True)
    var_names = w_dict.keys()
    var_list = [w_dict[kk] for kk in var_names]
    grads = tf.gradients(loss_a, var_list, gate_gradients=gate_gradients)

    var_list_new = [vv - gg for gg, vv in zip(grads, var_list)]
    w_dict_new = dict(zip(var_names, var_list_new))
    _, loss_b, logits_b = get_model(
        inp_b, label_b, ex_wts=ex_wts_b, is_training=True, reuse=True, w_dict=w_dict_new)
    grads_ex_wts = tf.gradients(loss_b, [ex_wts_a], gate_gradients=gate_gradients)[0]
    ex_weight = -grads_ex_wts
    ex_weight_plus = tf.maximum(ex_weight, eps)
    ex_weight_sum = tf.reduce_sum(ex_weight_plus)
    ex_weight_sum += tf.cast(tf.equal(ex_weight_sum, 0.0), tf.float32)
    ex_weight_norm = ex_weight_plus / ex_weight_sum
    return ex_weight_norm

